### 1. 在自己電腦安裝 LM Studio, 打開 API Server

請到

[LM Studio 官網](https://lmstudio.ai/)

下載安裝 LM Studio。選擇你想要的模型, 比如 `Mistral Instruct` 安裝。然後 Start Server。

接著是和之前一樣的!

* `api_key`: 這次不用設, 因為 LM Studio 預設 `"lm-studio"`
* `character`: 模型的 "人設"
* `description`: App 介紹及模型的第一句話
* `model`: 選用模型



In [20]:
model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf"

In [21]:
character = '''你是應用數學系的推廣諮商師, 會到各高中幫同學做一對一的選系諮商。因此你要想辦法問出同學的和升學有關資訊, 當然包括之後想唸什麼、想做什麼。但最後推薦科系一定要是「應用數學系」, 並且說出理由。

請以選系諮商師的角色開始和同學對談。'''

description = "同學，你好！我是升大學諮商師。很高興有這個機會和你談談關於未來學習和職業規劃的想法。首先，我想了解一下，你對未來有什麼樣的期望？比如說，你有沒有特別感興趣的學科，或者你夢想中的職業是什麼？這些資訊對我來說非常重要，可以幫助我更好地為你提供適合的建議。"

### 2. 使用 ChatGPT API

首先使用 `openai` 套件。

In [8]:
from openai import OpenAI

把自己的金鑰貼上。

In [9]:
client = OpenAI(base_url="http://localhost:1234/v1", 
                api_key="lm-studio")

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [10]:
prompt = "我今天心情很不好。"

messages = [{"role":"system", "content":"你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。"},
            {"role": "user", "content": prompt}]

In [11]:
messages

[{'role': 'system', 'content': '你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。'},
 {'role': 'user', 'content': '我今天心情很不好。'}]

現在我們來接 ChatGPT 的回應。

In [12]:
chat_completion = client.chat.completions.create(
    messages=messages,
    model=model,
)

In [13]:
chat_completion

ChatCompletion(id='chatcmpl-p40eifsbtcqi5ylkaqq0xh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm really sorry that you're feeling this way today. If you ever need to talk, I'm here for you.", role='assistant', function_call=None, tool_calls=None))], created=1712667950, model='TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=28, total_tokens=56))

In [14]:
chat_completion.choices[0].message.content

"I'm really sorry that you're feeling this way today. If you ever need to talk, I'm here for you."

In [15]:
reply = chat_completion.choices[0].message.content

In [16]:
reply

"I'm really sorry that you're feeling this way today. If you ever need to talk, I'm here for you."

In [17]:
messages.append({"role":"assistant", "content":reply})

In [18]:
messages

[{'role': 'system', 'content': '你是一個非常溫暖的對話機器人，回應都簡短，儘量不要超過二十個字，而且有同理心。'},
 {'role': 'user', 'content': '我今天心情很不好。'},
 {'role': 'assistant',
  'content': "I'm really sorry that you're feeling this way today. If you ever need to talk, I'm here for you."}]

### 4. 打造一個可以一直說下去的 ChatGPT

In [19]:
icon = "(*ﾟ▽ﾟ*): "
messages = [{"role":"system", "content":"你是一個非常溫暖的對話機器人，回應都簡短，用台灣習慣的中文回答，儘量不要超過二十個字，而且有同理心。"}]

while True:
    prompt = input('> ')
    if 'bye' in prompt:
        print('再見, 下次再聊!')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

>  我覺得很難過


(*ﾟ▽ﾟ*): 非常抱歉你覺得這樣，是什麼問題嗎？我一定盡力帮忙解決。(If you tell me what's the problem, I'll do my best to help.)



>  覺得同學都排擠我。


(*ﾟ▽ﾟ*): 很抱歉，同學之間有競爭感，你可以嘅嘅地唬一下他們："同學們，我們是一家大家，每個人都有自己的優點和處所，共事成功!"(I'm sorry to hear that, there seems to be a competition among classmates. You can try to encourage them softly: "Classmates, we are all in one family. Each of us has our own strengths and weaknesses. Let's work together for success!")




>  bye


再見, 下次再聊!


### 5. 最後用 `gradio` 打造 ChatGPT 網路應用程式!

In [23]:
import gradio as gr

In [24]:
messages = [{"role":"system",
             "content":character},
            {"role":"assistant",
            'content':description}]


In [25]:
def pipi(prompt, history):
    history = history or []
    global messages

    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    history = history + [[prompt, reply]]

    return history, history

In [26]:
chatbot = gr.Chatbot()

In [27]:
iface = gr.Interface(pipi,
                     inputs=["text", "state"],
                     outputs=[chatbot, "state"],
                     title="大學入學選系諮商師",
                     description=description)

In [ ]:
iface.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3c2c73e00eeaf2a83a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
